## Spatial Join Parcel/Scag to Census Data: 2010

In [91]:
# import libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.inspection import PartialDependenceDisplay

First we import and clean up the Parcel dataset that was created in LINK TO ALYSSA'S NOTEBOOK HERE

In [92]:
# read in data
parcel_data=pd.read_csv('data/join_scag_to_parcels_left_2019.csv')

# data clean-up
# use lambda functions to denote whether warehouse had been built on each parcel by 2010 and by 2020 (fixing error in previous version)
parcel_data['warehouse_2010']=parcel_data.year.apply(lambda x: 1 if x<2010 else 0)
parcel_data['warehouse_2020']=parcel_data.year.apply(lambda x: 1 if x<2020 else 0)
parcel_data['built_2010s']=parcel_data.year.apply(lambda x: 1 if (2010 <= x < 2020) else 0)

# replace county names w/ dummies
dummies=pd.get_dummies(parcel_data.COUNTY, prefix='county')
parcel_data = parcel_data.join(dummies)
parcel_data.head()

C:\Users\lucyb\AppData\Local\Temp\ipykernel_17168\984724337.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  parcel_data=pd.read_csv('data/join_scag_to_parcels_left_2019.csv')


,APN,LAND_VALUE,distances,building_class,year,acres,sqft,num_warehouses,warehouse_2010,warehouse_2020,...,Specific Plan,"Transportation, Communications, and Utilities",Under Construction,Undevelopable,Unknown,Vacant,Water,dollars_per_acre,county_Riverside,county_San Bernardino
0,933260003.0,48418.0,38449.033944,NaN,NaN,NaN,NaN,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2406.435207,1,0
1,933190003.0,210496.0,39355.337470,NaN,NaN,NaN,NaN,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10667.570219,1,0
2,933200001.0,347975.0,39133.409768,NaN,NaN,NaN,NaN,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,17690.981870,1,0
3,933180027.0,88468.0,39830.303362,NaN,NaN,NaN,NaN,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9505.916138,1,0
4,933180028.0,334555.0,38830.371787,NaN,NaN,NaN,NaN,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16032.116147,1,0


### Spatial Join to Census Data

In [109]:
#turning everything into float32 to save memory
for col in parcel_data.columns:
    if parcel_data[col].dtype == 'float64': # skip the string column (srprec)
        parcel_data[col] = parcel_data[col].astype('float16')
for col in parcel_data.columns:
    if parcel_data[col].dtype == 'int64': # skip the string column (srprec)
        parcel_data[col] = parcel_data[col].astype('int8')

In [94]:
#read in 2009 census data
census09 = gpd.read_file('data/census_2009.gpkg')

In [108]:
for col in census09.columns:
    if census09[col].dtype == 'float64': # skip the string column (srprec)
        census09[col] = census09[col].astype('float16')

In [106]:
#rejoining parcel geometry to make it into a Gdf
parcels_geometry = gpd.read_file('data/parcels_warehouses.gpkg', include_fields=['APN', 'geometry'])
#make APN into float32
parcels_geometry.APN = parcels_geometry.APN.astype('float16')

In [110]:
parcels_geometry.dtypes

APN          float16
geometry    geometry
dtype: object

In [107]:
parcel_data.dtypes

APN                                              float16
LAND_VALUE                                       float16
distances                                        float16
building_class                                    object
year                                             float16
acres                                            float16
sqft                                             float16
num_warehouses                                   float16
warehouse_2010                                      int8
warehouse_2020                                      int8
built_2010s                                         int8
lon                                              float16
lat                                              float16
PID19                                            float16
APN19                                            float16
COUNTY                                            object
CITY                                              object
ACRES                          

In [112]:
#drop old geometry column
#parcel_data = parcel_data.drop('geometry', axis = 1)
#joining back to scag/assessor data
parcel_data_geo = parcel_data.join(parcels_geometry.set_index('APN'), on = 'APN', how = 'left').reset_index()
parcel_data_geo = gpd.GeoDataFrame(parcel_data_geo, geometry = 'geometry')

MemoryError: Unable to allocate 14.1 TiB for an array with shape (1938749910240,) and data type int64

First we create a dataset to represent the conditions in the Inland Empire in 2010

In [ ]:
for col in parcel_data_geo.columns:
    if parcel_data_geo[col].dtype == 'int64': # skip the string column (srprec)
        parcel_data_geo[col] = parcel_data_geo[col].astype('int8')

In [47]:
#parcel_data_geo.to_file('parcel_scag_geo.gpkg', driver = 'gpkg')

In [ ]:
parceldata_10=parcel_data_geo.copy()
for col in parceldata_10.columns:
    if parceldata_10[col].dtype == 'float32': # skip the string column (srprec)
        parceldata_10[col] = parceldata_10[col].astype('float16')

In [ ]:
# copy dataset and delete/rename variables as needed
parceldata_10.drop(columns=['warehouse_2020'],inplace=True)
parceldata_10.rename(columns={'warehouse_2010':'warehouse_start','built_2010s':'buit_within_decade'},inplace=True)

In [ ]:
#spatial join to just the 2010 census data, use CA Zone 5 CRS 2229
parceldata_10 = parceldata_10.to_crs(2229)
census09 = census09.to_crs(2229)
parcelcensus10 = gpd.sjoin(parceldata_10, census09, how = 'left', predicate='intersects')

# need to set APN to be index so that we can join this back to other information later on
parcelcensus10.set_index('APN',inplace=True)

In [ ]:
print(parceldata10.shape)
print(parcelcensus10.shape)

In [ ]:
for col in parcelcensus10.columns:
    if parcelcensus10[col].dtype == 'int64': # skip the string column (srprec)
        parcelcensus10[col] = parcelcensus10[col].astype('int8')

In [21]:
#parcelcensus10.to_csv('data/parcelcensus10.csv')

In [ ]:
for col in parcelcensus10.columns:
    if parcelcensus10[col].dtype == 'float32': # skip the string column (srprec)
        parcelcensus10[col] = parcelcensus10[col].astype('float16')
for col in parcelcensus10.columns:
    if parcelcensus10[col].dtype == 'float64': # skip the string column (srprec)
        parcelcensus10[col] = parcelcensus10[col].astype('float16')
parcelcensus10.info()

In [ ]:
# define variables 
cols=parcelcensus10.columns.to_list()
xvars=[col for col in cols if col not in ('APN','building_class','year','acres','sqft','num_warehouses','buit_within_decade',
                                          'geometry','PID19','APN19','CITY','COUNTY','LU19','LU16','JURISDICTI','LU19_CLASS','SCAG_ZN_CO','NAME',
                                          'Shape_Leng','Shape_Area')]
yvar = 'buit_within_decade'

# create a dataframe with no NaNs
parceldata_10_model = parcelcensus10[xvars+[yvar]].dropna()

# create train-test split
X_train, X_test, y_train, y_test = train_test_split(
    parceldata_10_model[xvars], parceldata_10_model[yvar], test_size = 0.25, random_state = 1)

In [ ]:
# initialize the random forest classifer object
rf = RandomForestClassifier(n_estimators = 10, random_state = 1)

# fit the model
rf.fit(X_train, y_train)

# apply predictions to test dataset
y_pred = rf.predict(X_test)

In [ ]:
# stop if the length of the predictions doesn't match the training dataset
assert len(X_test)==len(y_pred)

We used a Confusion Matrix for an initial assessment of performance

In [ ]:
print('Predicted fraction True: {:.4f}. Actual fraction True: {:.4f}'.format(
    y_pred.mean(), y_test.mean()))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

We used the feature importances to identify variables worth looking into further

In [ ]:
# create series of importances
importances = rf.feature_importances_
forest_importances = pd.Series(importances, index=X_train.columns)
forest_importances.sort_values(inplace=True, ascending=False)

# plot them
fig, ax = plt.subplots(figsize=(5,4))
sns.barplot(x=forest_importances[:15].values, y=forest_importances[:15].index, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")